#Value-at-Risk

The included jupyter notebook describes an example of calculating value at risk for a given portfolio for a given, single asset such as the S&P 500. The user provides basic information such as the Asset (ticker symbol), the Years of historical data you wish to use, the Expected Percent Change that you want to measure, and the Value of the Portfolio you are estimating on.

The Python code retrieves asset data for a given number of years from IEX, using pandas datareader. It gives the option to export the data retrieved to a csv on a local machine titled "modified_data.csv".

The retrieved data has been modified to include the percentage change from day to day. A distribution of these percentage changes are then analyzed to compute the value at risk.

Two figures are provided, a histogram and gaussian curve, to help visualize the distribution of the percent daily change. A single vertical line is shown on the gaussian curve to distinguish the location on the distribution of the expected percent change.

In conclusion, a quantitative analysis of value-at-risk is given, along with the expected 1% risk of the portfolio in relation to the asset.

# Value at Risk Calculation System
This is a project I wrote for a graduate-level financial risk management course.There are three steps:

## Step 1
Implemented EWMA model to estimate daily volatilities of USD-MXN exchange rate returns from 2015 to 2017.

## Step 2
Introduced Monte Carlo simulation, parametric and historical methods to calculate one-day VaR of Mexican peso.

## Step 3
Back-tested VaR model for accuracy at different confidence levels using Chi-Square goodness-of-fit test.

# Step1 Estimate Daily Volatilities with EWMA Model¶

In [0]:
# this function wrote for calculating daily volatilities
# using EWMA (Exponentially Weighted Moving Average) model


def ewma_vol(data, observed_days, decay_factor):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    output type: dataframe(dataframe contains date, price, log-return and volatilitis)
    """
    # set the first vol using root mean square of return
    data.loc[(observed_days - 1), 'Volatility'] = np.sqrt(
        ((data.loc[:observed_days - 1, 'Log Return'] - 0) ** 2).mean())
    # calculate vol using EWMA recursive formula
    for index in range(observed_days, len(data)):
        data.loc[index, 'Volatility'] = np.sqrt(
            (1 - decay_factor) * data.loc[index, 'Log Return'] ** 2 + decay_factor * data.loc[(index - 1), 'Volatility'] ** 2)
    # drop the rows for null value
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    return data

# Step2 Calculate One-Day VaR
## 2.1 Monte Carlo Simulation

In [0]:
# use random draws from a hypothesized return distribution


def mc_simulation_var(data, observed_days, decay_factor, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and mc-simulation-var)
    """
    # calculate the EWMA volatility
    data = ewma_vol(data, observed_days, decay_factor)

    # use monte carlo simulation to create the var
    # calculate the var using the quantile of 1000 simulation
    for index in range(len(data)):
        random_returns = np.random.normal(
            loc=0, scale=data.loc[index, 'Volatility'], size=1000)
        var_return = np.percentile(
            random_returns, (1 - confidence_level) * 100)
        data.loc[index, 'VaR'] = data.loc[index,
                                          'Price'] * (1 - np.exp(var_return))

    return data


## 2.2 Parametric Method¶

In [0]:
# based on a hypothesized return distribution plus a volatility estimate


def parametric_var(data, observed_days, decay_factor, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and parametric-var)
    """
    # calculate the EWMA volatility
    data = ewma_vol(data, observed_days, decay_factor)
    # use parametric method to create the var
    data['VaR'] = data['Price'] * (1 - np.exp(norm.ppf(
        1 - confidence_level, loc=0, scale=data['Volatility'])))

    return data

## 2.3 Historical Method¶

In [0]:
# based on historical returns over some past observation period
# no distributional hypothesis


def historical_var(data, observed_days, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and historical-var)
    """

    for index in range(observed_days - 1, len(data)):
        data.loc[index, 'VaR'] = data.loc[index, 'Price'] * (1 - np.exp(np.percentile(
            data['Log Return'][index - observed_days + 1:index], (1 - confidence_level) * 100)))

    # drop the rows for null value
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    return data

# Step3 Back-Tested VaR Model with Chi-Squared Test¶

In [0]:
# use the Chi-Square goodness-of-fit test to back-test the VaR model


def backtest_var(data_var, confidence_level, confidence_level_chi2):
    """
    data type: dataframe(dataframe contains date, price, log-return and VaR)
    confidence_level type: float(confidence_interval for VaR)
    confidence_level_chi2 type: float(confidence_interval for chi-squared test)
    output type: dictionary(dictionary contains chi-squred test results)
    """
    # calculate the number of exceed observations
    exceed_num = len(
        data_var[(data_var['Price'].shift(1) - data_var['Price']) > data_var['VaR']])
    # calculate the total observations
    total_num = len(data_var) - 1
    # calculate the test statistics
    test_statistics = 2 * (np.log((exceed_num / total_num)**exceed_num * (1 - exceed_num / total_num)**(
        total_num - exceed_num)) - np.log((1 - confidence_level)**exceed_num * confidence_level**(total_num - exceed_num)))
    # calculate the 0.95 chi-squared statistics
    chi_squared_statistics = chi2.ppf(confidence_level_chi2, 1)

    # create the dictionnary to diaplay the results
    test_result = {}
    test_result['VaR Confidence-level'] = confidence_level
    test_result['Chi2 Confidence-level'] = confidence_level_chi2
    test_result['Exceed Num'] = exceed_num
    test_result['Total Num'] = total_num
    test_result['Test Statistics'] = test_statistics
    test_result['Chi-squared Statistics'] = chi_squared_statistics
    if test_statistics < chi_squared_statistics:
        test_result['Null Hypothesis'] = 'Accept'
    else:
        test_result['Null Hypothesis'] = 'Reject'

    return test_result

# Value-at-Risk Calculation System¶
## Import Library

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import chi2

## Main Function¶

In [0]:
# this is the main function of this project
# use lognormal and zero-drift model


def main_var(price_file_name, method, observed_days, decay_factor, confidence_level, confidence_level_chi2):
    """
    price_file_name type: str(name of the csv file containing date and price)
    method type: str(name of the method to calculate VaR)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    confidence_level type: float(confidence_level for VaR)
    confidence_level_chi2 type: float(confidence_interval for chi-squared test)
    output type: print results
    """

    # import the data from the csv file into a dataframe
    data = pd.read_csv(price_file_name)
    # calculate the log-return of the price
    data['Log Return'] = np.log(data['Price'] / data['Price'].shift(1))
    # drop the rows for null value
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    # monte carlo simulation method and back-testing
    if method == 'Monte Carlo Simulation':
        print('\nMonte Carlo Simulation')
        print('------------------------\n')

        var = mc_simulation_var(data, observed_days,
                                decay_factor, confidence_level)
        print('\nDisplay First Five Daily VaR\n')
        print(var.head(5))

        backtest_result = backtest_var(
            var, confidence_level, confidence_level_chi2)
        print('\nDisplay Backtest Result\n')
        print(backtest_result)

    # parametric method and back-testing
    if method == 'Parametric Method':
        print('\nParametric Method')
        print('------------------------\n')

        var = parametric_var(
            data, observed_days, decay_factor, confidence_level)
        print('\nDisplay First Five Daily VaR\n')
        print(var.head(5))

        backtest_result = backtest_var(
            var, confidence_level, confidence_level_chi2)
        print('\nDisplay Backtest Result\n')
        print(backtest_result)

    # historic method and back-testing
    if method == 'Historical Method':
        print('\nHistorical Method')
        print('------------------------\n')

        var = historical_var(data, observed_days, confidence_level)
        print('\nDisplay First Five Daily VaR\n')
        print(var.head(5))

        backtest_result = backtest_var(
            var, confidence_level, confidence_level_chi2)
        print('\nDisplay Backtest Result\n')
        print(backtest_result)

### Results

In [61]:
main_var('/content/mxn_usd.csv', 'Monte Carlo Simulation', 22, 0.94, 0.95, 0.95)



Monte Carlo Simulation
------------------------


Display First Five Daily VaR

        Date     Price  Log Return  Volatility       VaR
0  1/30/2015  0.066769   -0.012226    0.006728  0.000704
1   2/2/2015  0.067260    0.007331    0.006766  0.000746
2   2/3/2015  0.068245    0.014539    0.007464  0.000856
3   2/4/2015  0.067145   -0.016260    0.008260  0.000937
4   2/5/2015  0.067623    0.007095    0.008195  0.000932

Display Backtest Result

{'VaR Confidence-level': 0.95, 'Chi2 Confidence-level': 0.95, 'Exceed Num': 33, 'Total Num': 523, 'Test Statistics': 1.7503294818201027, 'Chi-squared Statistics': 3.841458820694124, 'Null Hypothesis': 'Accept'}


In [62]:
main_var('mxn_usd.csv', 'Parametric Method', 22, 0.94, 0.95, 0.95)



Parametric Method
------------------------


Display First Five Daily VaR

        Date     Price  Log Return  Volatility       VaR
0  1/30/2015  0.066769   -0.012226    0.006728  0.000735
1   2/2/2015  0.067260    0.007331    0.006766  0.000744
2   2/3/2015  0.068245    0.014539    0.007464  0.000833
3   2/4/2015  0.067145   -0.016260    0.008260  0.000906
4   2/5/2015  0.067623    0.007095    0.008195  0.000905

Display Backtest Result

{'VaR Confidence-level': 0.95, 'Chi2 Confidence-level': 0.95, 'Exceed Num': 31, 'Total Num': 523, 'Test Statistics': 0.8960549322980285, 'Chi-squared Statistics': 3.841458820694124, 'Null Hypothesis': 'Accept'}


In [63]:
main_var('mxn_usd.csv', 'Historical Method', 22, 0.94, 0.95, 0.95)



Historical Method
------------------------


Display First Five Daily VaR

        Date     Price  Log Return       VaR
0  1/30/2015  0.066769   -0.012226  0.000616
1   2/2/2015  0.067260    0.007331  0.000815
2   2/3/2015  0.068245    0.014539  0.000827
3   2/4/2015  0.067145   -0.016260  0.000814
4   2/5/2015  0.067623    0.007095  0.000822

Display Backtest Result

{'VaR Confidence-level': 0.95, 'Chi2 Confidence-level': 0.95, 'Exceed Num': 52, 'Total Num': 523, 'Test Statistics': 21.15789964671228, 'Chi-squared Statistics': 3.841458820694124, 'Null Hypothesis': 'Reject'}
